In [ ]:
import pandas as pd
import sweetviz as sv

In [ ]:
df = pd.read_csv(
    'data/raw/IHME_PHMRC_VA_DATA_ADULT_Y2013M09D11_0.csv',
    dtype={
        'g1_06d': str,
        'g2_03ad': str,
        'g2_03dd': str,
        'g2_03ed': str,
        'g5_03d': str,
        'a2_09_2b': str
    }
)

# remove columns that begin with "word_"
# these are BOW features from the open narratives
df = df[[c for c in df.columns if not c.startswith('word_')]]

# remove columns that begin with "c_"
# these are child related questions
df = df[[c for c in df.columns if not c.startswith('c_')]]

# map "don't know" to a numeric value to keep the dtype as integer
def convert_dont_know_to_int(x):
    if isinstance(x, str):
        if (x.strip() == "Don't Know"):
            return -1
        else:
            return pd.to_numeric(x)
    else:
        return x

integer_columns = [
    'g1_01d',
    'g1_01y',
    'g1_06d',
    'g1_06y',
    'g1_07a',
    'g1_07b',
    'g1_07c',
    'g1_10',
    'g2_03ad', # Date of first interview attempt [day]
    'g2_03ay', # Date of first interview attempt [year]
    'g2_03bd', # Date and time arranged for second interview attempt [day]
    'g2_03by', # Date and time arranged for second interview attempt [year]
    'g2_03cd', # Date and time arranged for third interview attempt [day]
    'g2_03cy', # Date and time arranged for third interview attempt [year]
    'g2_03dd', # Date of interview [day]
    'g2_03dy', # Date of interview [year]
    'g2_03ed',
    'g2_03ey',
    'g2_03fd',
    'g2_03fy',
    'g5_01d', # When was the deceased born? [day]
    'g5_01y', # When was the deceased born? [year]
    'g5_03d', # When did he/she die? [day]
    'g5_03y', # When did he/she die? [year]
    'g5_04a', # What was the last known age of the deceased? [years]
    'g5_04c', # What was the last known age of the deceased? [days]
]

for c in integer_columns:
    df[c] = df[c].map(convert_dont_know_to_int).astype('Int64')

print('Consent:')
print(df['g3_01'].value_counts())
df.head()

## Load in the free-text data

In [ ]:
text_codebook = pd.read_excel('data/raw/PHMRC_VAI_redacted_free_text.xlsx', sheet_name=0)
text = pd.read_excel('data/raw/PHMRC_VAI_redacted_free_text.xlsx', sheet_name=1)

# subselect to the adult module
text = text.loc[text['module'] == 'Adult']

# newid is only unique when scoped by adult/child/neonate
newid_multiple_row = text['newid'].value_counts()
newid_multiple_row = newid_multiple_row.index[newid_multiple_row>1]
assert len(newid_multiple_row) == 0, 'found multiple row for newid, check only adults included'

print(text.shape)
text.head()

## Apply exclusions / data cleaning

In [ ]:
print(f'{df.shape[0]} rows in primary dataframe.')
dff = df.merge(text, how='left', on=['newid', 'module'], suffixes=('', '_text'))

print(f'{dff.shape[0]} rows after merge.')
dff['age_years'] = dff['age_years'].astype('Int64')

# compare columns
idx = dff['open_response'].isnull()
print(f'  {idx.sum()} rows missing text data.')
dff = dff.loc[~idx]

idx = (dff['g5_06a'].isnull()) | (dff['g5_06a'] == 'Unknown')
print(f'  {idx.sum()} rows missing education data.')
dff = dff.loc[~idx]

for c in ['site', 'gs_text34']:
    idx = (dff[c] == dff[f'{c}_text']) & (dff[f'{c}_text'].notnull())
    print(f'{idx.sum()} / {dff[f"{c}_text"].notnull().sum()} rows match for {c}.')

# subselect data
columns = [
    'site', # Site
    'newid', # Study ID
    # free-text columns
    'sex',
    'age_years',
    'age_months',
    'age_days',
    'gs_text34',
    'open_response',
    
    # the diagnosis vars listed in the codebook look inconsistent with the dataframe
    # ... so we are ignoring them, but the df cols are listed below
    # 'gs_code34', 'gs_text34', 'va34'
    # 'gs_code46', 'gs_text46', 'va46', 
    # 'gs_code55', 'gs_text55', 'va55',
    
    # for reference, we expected the following based on the codebook:
    # 'gs_diagnosis', # Gold Standard Diagnosis Code
    # 'gs_comorbid1', # Gold Standard Comorbid Conditions 1
    # 'gs_comorbid2', # Gold Standard Comorbid Conditions 2
    # 'gs_level', # Gold Standard Diagnosis Level
    
    # questionnaire
    'g1_01d', # Date of birth [day]
    'g1_01m', # Date of birth [month]
    'g1_01y', # Date of birth [year]
    'g1_05', # Sex of deceased
    'g1_06d', # Date of death [day]
    'g1_06m', # Date of death [month]
    'g1_06y', # Date of death [year]
    'g1_07a', # Last known age of the deceased [years]
    'g1_07b', # Last known age of the deceased [months]
    'g1_07c', # Last known age of the deceased [days]
    'g1_08', # Marital status of deceased
    'g1_09', # Last known level of education completed
    'g1_10', # Number of years of education completed
    'g2_01', # Language of interview
    'g2_02', # Interviewer ID number
    'g2_03ad', # Date of first interview attempt [day]
    'g2_03am', # Date of first interview attempt [month]
    'g2_03ay', # Date of first interview attempt [year]
    'g2_03bd', # Date and time arranged for second interview attempt [day]
    'g2_03bm', # Date and time arranged for second interview attempt [month]
    'g2_03by', # Date and time arranged for second interview attempt [year]
    'g2_03cd', # Date and time arranged for third interview attempt [day]
    'g2_03cm', # Date and time arranged for third interview attempt [month]
    'g2_03cy', # Date and time arranged for third interview attempt [year]
    'g2_03dd', # Date of interview [day]
    'g2_03dm', # Date of interview [month]
    'g2_03dy', # Date of interview [year]
    'g3_01', # Did respondent give consent?
    'g4_02', # What is the sex of the respondent?
    'g4_03a', # What is your respondent's relationship to the deceased?
    'g4_03b', # Other relationship specified
    'g4_04', # How old are you?
    'g4_05', # # "What was the last level of school you attended?" -> respondent education level
    'g5_01d', # When was the deceased born? [day]
    'g5_01m', # When was the deceased born? [month]
    'g5_01y', # When was the deceased born? [year]
    'g5_02', # What was the sex of the deceased?
    'g5_03d', # When did he/she die? [day]
    'g5_03m', # When did he/she die? [month]
    'g5_03y', # When did he/she die? [year]
    'g5_04a', # What was the last known age of the deceased? [years]
    'g5_04b', # What was the last known age of the deceased? [months]
    'g5_04c', # What was the last known age of the deceased? [days]
    
    # outcome vars
    'g5_06a', # What was the last known level of education of deceased?
    'g5_06b', # Number of years of education completed
]

dff = dff[columns]

# apply mapping for sex from codebook
# 1 "Male" 2 "Female" 8 "Refused to Answer" 9 "Don't Know"
dff['sex'] = dff['sex'].map({1: 'Male', 2: 'Female', 8: 'Refused to Answer', 9: "Don't Know"})

# replace ages of 999 with -1
dff.loc[dff['age_years'] == 999, 'age_years'] = -1

# 19 rows were discrepant by sex after merging, so were deleted
def print_discrepency(dff, var1, var2, mapping=None):
    """transform allows mapping vars in var1"""
    v1 = dff[var1].copy()
    v2 = dff[var2].copy()
    if mapping is not None:
        v1 = v1.map(mapping)
    
    print(f'=== {var1} vs {var2} ===')
    idx1 = v1.notnull()
    idx2 = v2.notnull()
    print(f'  {var1}: {idx1.sum()} rows, {dff.loc[idx1, var1].nunique()} unique values.')
    print(f'  {var2}: {idx2.sum()} rows, {dff.loc[idx2, var2].nunique()} unique values.')
    idx = idx1 & idx2
    print(f'  Overlap: {idx.sum()} rows, with {(v1[idx] != v2[idx]).sum()} mismatches.')

print_discrepency(dff, 'sex', 'g5_02')
print_discrepency(dff, 'age_years', 'g5_04a')

# 1408 rows were discrepant by age but were clearly correlated when analysed by scatter plot, so we chose to keep age from the gold standard dataset.
# Highest educational level was discrepant for 1977 rows between what was reported by the interviewee and what was formally recorded in the decedent’s record;
# in these cases we selected the highest educational 103 level between the two.
dff.set_index('newid', inplace=True)
print(dff.shape)
dff.head()

## Output to file

In [ ]:
dff.to_csv('data/processed/merged_dataset.csv')